##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Загрузка pandas.DataFrame

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/pandas_dataframe"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ru/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ru/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/ru/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Из этого руководства вы узнаете, как загрузить датафрейм Pandas в `tf.data.Dataset`.

Это руководство использует небольшой [датасет](https://archive.ics.uci.edu/ml/datasets/heart+Disease) предоставленный Фондом сердечных заболеваний кливлендской клиники. В ней несколько сотен строк в формате CSV. Каждая строка описывает пациента, а каждый столбец описывает свойство. Мы используем эту информацию чтобы предсказать имеет ли пациент сердечное заболевание, что в этом датасете является задачей двоичной классификации.

## Прочитайте данные используя pandas

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version существует только в Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import pandas as pd
import tensorflow as tf

Скачайте файл csv содержащий датасет с пациентами.

In [0]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')

Прочитайте csv файл используя pandas.

In [0]:
df = pd.read_csv(csv_file)

In [0]:
df.head()

In [0]:
df.dtypes

Конвертируйте столбец `thal` являющийся `object` в датафрейме в дискретные числовые значения.

In [0]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [0]:
df.head()

## Загрузите данные используя `tf.data.Dataset`

Используйте `tf.data.Dataset.from_tensor_slices` чтобы прочитать значения из датафрейма pandas. 

Одним из преимуществ использования `tf.data.Dataset` является то, что он позволяет вам писать простые высокоэффективные конвейеры данных. Прочитайте [руководство по загрузке данных](https://www.tensorflow.org/alpha/guide/data) для подробностей.

In [0]:
target = df.pop('target')

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [0]:
for feat, targ in dataset.take(5):
  print ('Признаки: {}, Цель: {}'.format(feat, targ))

Поскольку `pd.Series` реализует протокол `__array__` он может быть использован практически везде где вы бы использовали `np.array` или `tf.Tensor`.

In [0]:
tf.constant(df['thal'])

Перемешайте датасет и разбейте его на пакеты.

In [0]:
train_dataset = dataset.shuffle(len(df)).batch(1)

## Создайте и обучите модель

In [0]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [0]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)

## Альтернатива столбцам признаков

Передать словарь в качестве входных данных для модели так же просто, как и создать словарь соответствия слоев `tf.keras.layers.Input`, применяя любой препроцессинг и складывая их, используя [функциональный api](../../guide/keras/functional.ipynb). Вы можете использовать это в качестве альтернативы [столбцов признаков](../keras/feature_columns.ipynb).

In [0]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)

x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

Простейший способ сохранения структуры столбцов в `pd.DataFrame` при использовании с `tf.data` это конвертация `pd.DataFrame` в `dict` и сделав срезы этого словаря.

In [0]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

In [0]:
for dict_slice in dict_slices.take(1):
  print (dict_slice)

In [0]:
model_func.fit(dict_slices, epochs=15)